In [1]:
import random
from bolib.Bo import Bo
from bolib.Dimension import NumericDimension
from bolib.ComputeSpace import ComputeSpace
from bolib.Normalizer import OptimizeFor
import pandas as pd
from IPython.display import clear_output
import numpy as np

c:\Users\phili\Documents\workspace\botorch\botorch_contrast\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
blackbox_dims = 6

In [3]:

def getRating(v, target):    
    diff = abs(np.array(v) - np.array(target))/(100*len(v))    
    return [sum(diff.tolist())]

def getSample(TARGET = None):
    s = [random.randint(0, 100) for _ in range(blackbox_dims)]
    if TARGET is not None:
        r= getRating(s, TARGET)
        return [s, r]
    return s


TARGET = getSample()

s, r = getSample(TARGET)
pd.DataFrame([[s, r, TARGET]], columns=["guess", "rating", "target"])  

,guess,rating,target
0,"[98, 88, 82, 91, 19, 4]",[0.47000000000000003],"[31, 64, 33, 56, 4, 96]"


In [4]:
numerics = [ NumericDimension(min=0, max=100, name=f"p{i}") for i in range(blackbox_dims) ]
ranking = [ NumericDimension(min=0, max=10, name="Ranking", optimize_for=OptimizeFor.MIN) ]

compSpace = ComputeSpace(numerics, ranking)


bo = Bo(compSpace)


def infer(x, y):  
  compSpace.add_values(xs = [x], ys=[y])   
  inf = bo.infer()
  denorm = compSpace.denormalize(inf)
  return [d.value for d in denorm]

infer(s,r)




[0.0, 100.0, 100.0, 0.0, 0.0, 100.0]

In [6]:
last_rating = 0
last_guess = None
best_guess = None
best_rating = None
print(f"We are looking for {TARGET}")

df_cols = ["guess", "rating", "target"]
df = pd.DataFrame([], columns=df_cols)  

for i in range(20):
  last_rating = r[0]
  last_guess = s
  
  
  guess = infer(s,r)  
  
  r = getRating(guess, TARGET)

  tdf = pd.DataFrame([[s, r, TARGET]], columns=df_cols)  
  
  df = pd.concat([df, tdf], axis=0).reset_index(drop=True)
  
  clear_output(wait=True)

  print(df)

  
  #if r[0] == 2 
  """
  if r[0] < 0.15:
    print(f"Found it in {i} steps: {guess}")
    break
  """
  if best_guess is None:
    best_rating = r[0]
    best_guess = guess

  if r[0] < best_rating:
    best_guess = guess
    best_rating = r[0]
  s = guess

print(f"I think it is {best_guess} with a rating of {best_rating}")


  


  



                                                guess                  rating  \
0                             [98, 88, 82, 91, 19, 4]    [0.3170258604983489]   
1   [0.0, 9.35378447175026, 5.933183431625366, 29....   [0.34630482616523894]   
2   [0.0, 0.0, 0.0, 5.420173332095146, 15.12966305...    [0.3967420387764772]   
3   [0.0, 0.0, 0.0, 15.490761399269104, 10.0828625...   [0.26130977059404054]   
4   [0.0, 15.60262143611908, 9.85119715332985, 38....   [0.21832191313306495]   
5   [0.0, 17.255176603794098, 6.865977495908737, 5...    [0.2104971808195114]   
6   [0.0, 19.345322251319885, 21.18784487247467, 6...    [0.2011361506084601]   
7   [0.0, 35.064512491226196, 9.30984541773796, 69...   [0.23681339363257092]   
8   [0.0, 31.22008740901947, 9.01688039302826, 65....   [0.18734449416399002]   
9   [9.657441824674606, 28.88655662536621, 11.5263...   [0.21068060884873072]   
10  [0.0, 26.23281180858612, 9.35424193739891, 83....   [0.14035168930888176]   
11  [7.957477867603302, 35.3